In [12]:
# Importe as bibliotecas necessárias

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import pandas_market_calendars as mcal
import math

import warnings
warnings.filterwarnings('ignore')

In [13]:
preLast = pd.read_excel("dados/Risco_Retorno_Base.xlsx",sheet_name=2,index_col = 0, header = 1)

In [14]:
# Declarando varíaveis de estudo

ativo = str(input("Selecione o ativo:  "))
print("")
print("-----------------------------------------")
data_inicial = str(input("Insira a data de aquisição (formato YYYY-MM-DD): "))

# Data final (para o trabalho será considerada 2023-03-31)
data_final = "2023-03-31"

# Data final para último dia útil
#cal = mcal.get_calendar('B3')
#today = date.today()
#last_business_day = cal.schedule(start_date=today - timedelta(days=365), end_date=today).index[-1].date()
#data_final = last_business_day.strftime('%Y-%m-%d')

KeyboardInterrupt: Interrupted by user

In [ ]:
# Organizando o DataFrame preLast

# Criando um Dataframe preLast_1
preLast_1 = pd.DataFrame() # Contém o Índice Bovespa e Índice DI

# Remover as colunas .BVSP e INDEX_DI do primeiro dataframe e adicioná-las ao segundo dataframe
preLast_1.insert(loc=len(preLast_1.columns), column='.BVSP', value=preLast.pop('.BVSP'))
preLast_1.insert(loc=len(preLast_1.columns), column='DI_INDEX', value=preLast.pop('DI_INDEX'))

In [ ]:
# Dados Financeiro

# declarando capital próprio e de terceiros
capital_proprio = 1000
capital_terceiro = 1000
capital_total = capital_proprio + capital_terceiro

# Declarando a taxa de juros a.a e cálculo da taxa equivalente a.d
taxa_juros_a_a = 0.12
taxa_juros_a_d = (1+taxa_juros_a_a) ** (1/252) - 1

# Composição de capital
peso_capital_proprio = round((capital_proprio / capital_total) * 100,2)
peso_capital_terceiro = round((capital_terceiro / capital_total) * 100,2)

# Calculo da alavancagem
alavancagem = round(peso_capital_terceiro / peso_capital_proprio,2)
exposicao = str(1 + alavancagem) + "x"

In [ ]:
# Dados da operação

preco_compra = 10
preco_venda = 12
numero_dias = 252
oscilacao_preco = round((preco_venda - preco_compra) / preco_venda * 100,2)

In [ ]:
# DataFrame para apresentação dos dados financeiros

df_dados_financeiro = pd.DataFrame({
    'Capital Próprio': [capital_proprio],
    'Capital de Terceiros': [capital_terceiro],
    'Taxa de juros ao a.a': [taxa_juros_a_a],
    'Taxa de juros ao a.d':[taxa_juros_a_d],
    'Capital Total':[capital_total],
    'Peso Capital Próprio': [peso_capital_proprio],
    'Peso Capital Terceiro':[peso_capital_terceiro],
}).T

df_dados_financeiro.columns = ["Dados Financeiros"]

In [ ]:
# Cálculo da Operação sem Alavancagem

# Calculo da quantidade de compra
quantidade = int(capital_proprio / preco_compra)

# Cálculo valor de compra e venda
valor_posicao_data_compra = quantidade * preco_compra
valor_posicao_data_venda = quantidade * preco_venda

# Calculo do resultado da operação
resultado_operacao_sem_alavancagem = valor_posicao_data_venda - valor_posicao_data_compra
retorno_operacao_sem_alavancagem = round(((valor_posicao_data_venda / capital_proprio) - 1)*100,2)



# Cálculo da Operação com Alavancagem

# Calculo da quantidade de compra
quantidade_alavancado = int(capital_total / preco_compra)

# Cálculo valor de compra e venda
valor_posicao_data_compra_alavancado = quantidade_alavancado * preco_compra
valor_posicao_data_venda_alavancado = quantidade_alavancado * preco_venda

# Cálculo dos juros do capital de terceiros
juros = round(capital_terceiro * (1 + taxa_juros_a_d) ** (numero_dias) - capital_terceiro,2)

# Calculo do resultado da operação
resultado_operacao_com_alavancagem = valor_posicao_data_venda_alavancado - valor_posicao_data_compra_alavancado

# Cálculo do montante
montante_alavancado = capital_total + resultado_operacao_com_alavancagem - capital_terceiro - juros

# Calculo do retorno - resultado da operação
retorno_operacao_com_alavancagem = round(((montante_alavancado / capital_proprio) - 1)*100,2)

# Cálculo do lucro / prejuízo
lucro = resultado_operacao_com_alavancagem - juros

# DataFrame para apresentação da operação
df_operação = pd.DataFrame({
    "Capital Próprio": [capital_proprio, capital_proprio],
    "Capital Terceiro":[0, capital_terceiro],
    "Capital total": [capital_proprio, capital_total],
    "Exposição": ['1.0x', exposicao],
    "Quantidade": [quantidade, quantidade_alavancado],
    "Valor Posição Data Compra": [valor_posicao_data_compra, valor_posicao_data_compra_alavancado],
    "Valor Posição Data Venda":[valor_posicao_data_venda, valor_posicao_data_venda_alavancado],
    "Resultado da Operação":[resultado_operacao_sem_alavancagem, resultado_operacao_com_alavancagem],
    "Devolução do Princial":[0, capital_terceiro],
    "Juros": [0, juros],
    "Montante final": [valor_posicao_data_venda, montante_alavancado],
    "Lucro / Prejuízo": [resultado_operacao_sem_alavancagem,lucro],
    "Retorno sobre o K próprio (%)": [retorno_operacao_sem_alavancagem, retorno_operacao_com_alavancagem]
}).T

df_operação.reset_index()
df_operação.columns = ['Sem Alavancagem', 'Com Alavancagem']

In [ ]:
# Construção do DataFrame de dados da operação

df_dados_operacao = pd.DataFrame({
    'Preço de Compra': [preco_compra],
    'Preço de Venda': [preco_venda],
    'Número de dias': [numero_dias],
    'Oscilação de Preço': [oscilacao_preco],
    "Oscilação de Preço c/ Alavancagem": [retorno_operacao_com_alavancagem]
}).T

df_dados_operacao.columns = ['Dados da Operação']

In [ ]:
# Cálculo do Retorno esperado e Volatilidade do ativo 

# Cálculo sem alavancagem

# Quantidade comprada inicial
preco_compra_inicial = preLast[ativo].loc[data_inicial]
quant = capital_proprio / preco_compra_inicial

# Cálculo do retorno, retorno esperado e volatilidade
price = preLast[ativo].loc[data_inicial:data_final]
price_initial = price * quant
returns = np.log(price_initial / price_initial.shift(1)).dropna()
retorno_esperado = round((returns.mean() * 252)*100,2)
volatilidade = round((returns.std() * 252 ** 0.5)*100,2)


# Cálculo com alavancagem

# Quantidade comprada inicial
quant_alavancado = capital_total / preco_compra_inicial

# Cálculo do preço diário * quantidade
price_modificado_1 = price * quant_alavancado
  
# Cálculo da diferença de preço diário
price_ajustado =  price_modificado_1 - price_modificado_1.shift(1).dropna()

# Valor de juros diário referente ao capital de terceiros
juros_diario = taxa_juros_a_d * capital_terceiro

# Cálculo do valor diário líquido
valor_liquido = price_ajustado - juros_diario

# Cálculo do retorno, retorno esperado e volatilidade considerando o valor diário líquido
# OBS.: Nesta parte que é considerado os retornos alavancados 
price_initial_1 = price * quant_alavancado
returns_oficial = np.log(price_initial_1 / price_initial_1.shift(1)).dropna()
retorno_esperado_oficial = round((returns_oficial.mean() * 252)*100,2)
volatilidade_oficial = round((returns_oficial.std() * 252 ** 0.5) * 100,2)


In [ ]:
# Organizando as informações no DataFrame para a plotagem do gráfico

df_operacao_2 = pd.DataFrame()
df_operacao_2["Sem Alavancagem"] = price_sucessora
df_operacao_2["Com Alavancagem"] = price_initial_1

In [ ]:
# Cálculo para condição

# Retorno Esperado
try:
    resultado_1 = retorno_esperado_oficial
except:
    resultado_1 = "Quebrou"

# Desvio Padrão
try:
    resultado_2 = volatilidade_oficial
except:
    resultado_2 = "Quebrou"
    
# Resultado da alavancagem
try:
    resultado_3 = str(math.floor(retorno_esperado_oficial / retorno_esperado * 10) / 10) + "x"
except:
    resultado_3 = "Quebrou"
    
try:
    resultado_4 = str(math.floor(volatilidade_oficial / volatilidade * 10) / 10) + "x"
except:
    resultado_4 = "Quebrou"    

In [ ]:
# Organizando o Dataframe para a plotagem da tabela

df_tabela=pd.DataFrame({
    "Ln Retorno anualizado": [retorno_esperado, resultado_1, resultado_3],
    "Desvio Padrão anualizado": [volatilidade, resultado_2, resultado_4],
}).T

df_tabela.columns = [f"{ativo} sem Alavancado", f"{ativo} com alavancagem", 'Resultado da Alavancagem']

## Alavancagem - apresentação

## Dados Financeiros

In [ ]:
df_dados_financeiro

## Dados da Operação

In [ ]:
df_dados_operacao

## Resumo da Operação

In [ ]:
df_operação

## Visualização Gráfica

In [ ]:
# Construção do Gráfico
fig = make_subplots(rows = 1, cols = 1,
                   shared_xaxes = True,
                   vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = df_operacao_2.index
                        , y = df_operacao_2["Sem Alavancagem"]
                        , name = "sem alavancagem"
                        , line = dict(color = "black"))
             , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df_operacao_2.index
                        , y = df_operacao_2["Com Alavancagem"]
                        , name = "com alavancagem"
                        , line = dict(color = "red"))
             , row = 1, col = 1)

fig.update_layout(height = 600, width = 800
                 , title_text = f"{ativo} - Sem Alavancagem x Com Alacangagem"
                 , font_color = "blue"
                 , title_font_color = "black"
                 , xaxis_title = "Tempo"
                 , yaxis_title = "R $"
                 , showlegend = False
                 , legend_title = "Carteira"
                 , font = dict(size = 15, color = "Black")
                 
                 )
fig.update_layout(hovermode = "x")
fig.show()

In [ ]:
df_tabela